# OLS via Neural Network

Interpret linear regression as an extremely simple feedforward neural network.


In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import statsmodels.api as sm


The data generating process (DGP) simulates a linear relationship with three independent variables. 

1.  **Sample Size ($n$)**: 100 observations.
2.  **Independent Variables ($x_1, x_2, x_3$)**: Drawn independently from a standard normal distribution, $\mathcal{N}(0, 1)$.
3.  **Parameters**:
    *   Intercept: $\beta_0 = 0.5$
    *   Coefficients: $\beta_1 = 1$, $\beta_2 = -1$, $\beta_3 = 1$
4.  **Noise ($\epsilon$)**: The error term follows a standard normal distribution, $\epsilon \sim \mathcal{N}(0, 1)$.
5.  **Dependent Variable ($y$)**: Generated according to the equation $y = 0.5 + 1x_1 - 1x_2 + 1x_3 + \epsilon$.

In [4]:

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data with three regressors
n = 100  # number of observations
x1 = np.random.normal(0, 1, n)  # first explanatory variable
x2 = np.random.normal(0, 1, n)  # second explanatory variable
x3 = np.random.normal(0, 1, n)  # third explanatory variable
beta_0 = 0.5  # intercept
beta_1 = 1.  # coefficient for x1
beta_2 = -1.  # coefficient for x2
beta_3 = 1.  # coefficient for x3
epsilon = np.random.normal(0, 1, n)  # error term with standard normal distribution
y = beta_0 + beta_1 * x1 + beta_2 * x2 + beta_3 * x3 + epsilon  # dependent variable

# Create a DataFrame
data = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'y': y})


Use `sm.OLS()` as in statistics / econometrics.

In [5]:
# Fit OLS regression using statsmodels
# produce an X with `x1`, x2`, and `x3` in data
X = data[['x1', 'x2', 'x3']]
X = sm.add_constant(X)  # add a constant term to the model
y = data['y']

model = sm.OLS(y, X).fit()
print("OLS Regression Results Summary:")
print(model.summary())



OLS Regression Results Summary:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     142.8
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           2.84e-35
Time:                        22:04:17   Log-Likelihood:                -127.46
No. Observations:                 100   AIC:                             262.9
Df Residuals:                      96   BIC:                             273.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.587

A `pytorch` implemenation.

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Convert the data to PyTorch tensors
X_tensor = torch.tensor(data[['x1', 'x2', 'x3']].values, dtype=torch.float32)
y_tensor = torch.tensor(data['y'].values.reshape(-1, 1), dtype=torch.float32)

In [9]:
# Define a simple neural network with one hidden layer
class OLS_NN(nn.Module):
    def __init__(self):
        super(OLS_NN, self).__init__()
        self.output = nn.Linear(3, 1)  # three inputs, one output

    def forward(self, x):
        return self.output(x)  # identity function as activation


In [10]:
# Instantiate the model, define the loss function and the optimizer
model = OLS_NN()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [11]:
# Train the model
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 200 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')


Epoch 200/1000, Loss: 0.7498
Epoch 400/1000, Loss: 0.7493
Epoch 600/1000, Loss: 0.7493
Epoch 800/1000, Loss: 0.7493
Epoch 1000/1000, Loss: 0.7493


In [12]:
# Report the weights after training
print("\nWeights after training:")
for name, param in model.named_parameters():
    print(f"{name}: {param.data.numpy()}")


Weights after training:
output.weight: [[ 0.8226794 -1.0382409  1.0268975]]
output.bias: [0.58753484]


The estimated coefficients from NN are the same as from OLS.